In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 61.1MB/s 
     |████████████████████████████████| 3.8MB 40.1MB/s 
     |████████████████████████████████| 92kB 7.3MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


## 데이터를 불러와주고, 문자가 아닌 것들을 없애줌
    정규식을 활용하여 특정 문자가 아닌 것들을 없애주는 코드 작성

In [ ]:
import pandas as pd
import numpy as np
with open('/content/gdrive/My Drive/자연어기말/train_data.csv', 'r') as rt:
    train_df = pd.read_csv(rt).dropna()
with open('/content/gdrive/My Drive/자연어기말/test_data.csv', 'r') as et:
    test_df = pd.read_csv(et).dropna()
train_df['content'] = train_df['content'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") 
train_df['content'].replace('', np.nan, inplace=True)
train_df = train_df.dropna(how='any') 
train_df['title'] = train_df['title'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_df['title'].replace('', np.nan, inplace=True) 
train_df = train_df.dropna(how='any') 
test_df['content'] = test_df['content'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") 
test_df['content'].replace('', np.nan, inplace=True) 
test_df = test_df.dropna(how='any') 
test_df['title'] = test_df['title'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") 
test_df['title'].replace('', np.nan, inplace=True)
test_df = test_df.dropna(how='any') 

In [ ]:
train_df.head()

,category,content,title
0,정치개혁,문통 직접 민의를 살피지 않고 권력의 권좌에 앉아 보고만 받고 권력의 힘만 누렸으니...,문통 직접 민의를 살피지 않고 보고에만 의존하였으니 이젠 가망이 없소
1,육아/교육,초등학교 내 학부형 모임인 아버지회 의 문제점을 제기하고자 합니다 초등학교 내 아버...,뇌물수수 청탁이 만연하는 초등학교 아버지회를 없애주세요
2,보건복지,주변을 둘러보면 생활이 어렵지만 기초생활수급자 자격요건에 해당되지 않아 지원을 받지...,기초생활수급자 자격요건을 확대해주세요
3,경제민주화,거래소 노조 시감위원장 금융위 낙하산 인사 중단하라 한국거래소 노조는 현재 진행 중...,문재인 대통령님 시장감시위원장 인선과 관련해 금융위원회 낙하산 인사를 즉각 중단하십시요
4,육아/교육,안녕하세요 서울시 송파구 풍납동 어린이집 보낸 김여 정남아이 두 자녀의 부모님입니다...,저희아이들이 어린이집서 학대를 당했습니다 어린이집 아동학대 피해아동 구제해 주세요


In [ ]:
test_df.head()

,category,content,title
0,육아/교육,세 자녀 이상을 둔 다자녀 가정의 경우 대학생 국가장학금 제도 개편이 정말 시급합니...,다자녀 가정 대학 국가장학금 제도 개선해 주세요
1,정치개혁,국민을위해 국민을위한다면 정말 대한민국을 위한다면 자리에 연연하지마시고 그만 내려오세요,조국 이재명 임종석 탁현민 유은혜 이제 자리에서 그만 내려오세요
2,정치개혁,공무원은 죄가잇으면 처벌안합니가 업무상 죄가있으면 두배세배 처벌해야 되야되는거아닙니...,촛불은 힘있고 권력있는자 아니면 공무원을 위한촛불입니까
3,인권/성평등,현재 사회에서 가장 이슈가 되는것이 남녀간의 갈등이 가장 큰 이슈인걸 알고 있을실거...,남녀간의 갈등 대한민국 남성은 노예
4,정치개혁,문재인 대통령이 전정부 채동욱 전검찰총장 해임에 강력히 반발하던 특정지역 윤석열을 ...,공정수사을 위하여 윤석열 파면 하세요


## 전처리 과정
    특정 품사만 포함시켜주고 거기에 몇개의 스탑워드를 더 걸어줌
    스탑워드를 더 잘 설정하면 성능이 더 올라갈 것임 (시간이 없어서...)

In [ ]:
from konlpy.tag import Okt
okt = Okt()
yes = [ "Adjective", "Noun", "verb"]
stop_words = ["그","같다","있다", "좋다", "아니다", "등","좀"]
def clean_text(text):
    text = okt.pos(text, norm = True, stem = True)
    tmp = [t[0] for t in text if t[1] in yes and t[0] not in stop_words]
    return " ".join(tmp)
train_df["whole_pos"] = (train_df["content"] + train_df["title"]).map(clean_text)
test_df["whole_pos"] = (test_df["content"] + test_df["title"]).map(clean_text)

In [ ]:
train_df["whole_no"] = train_df["content"] + train_df["title"]
test_df["whole_no"] = test_df["content"] + test_df["title"]

In [ ]:
x_train, y_train = train_df["whole_pos"], train_df["category"]
x_test, y_test = test_df["whole_pos"], test_df["category"]

### 한글 -> 숫자 카테고리
    한글 카테고리를 숫자로 바꿔주는 기능 (원핫벡터로 바꾸기 전에 숫자로 먼저 바꿔준다)

In [ ]:
wd = {}
def to_cat(words):
    global wd
    wd ={}
    count = 0
    for i in set(words):
        wd[i] = count
        count += 1
    return_ = []
    for i in words:
        return_.append(wd[i])
    return np.array(return_)

y_train = to_cat(y_train)
y_test = to_cat(y_test)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, y_train, X_val, y_val = train_test_split(x_train, y_train, random_state = 10)

## 딥러닝 모듈 불러와서 학습시켜준다
    LSTM을 사용하였고, https://wikidocs.net/44249와 https://wikidocs.net/22933 에서 코드 아이디어를 얻어옴

    얻어온 아이디어와 한국어 여러 전처리 기술을 활용하여 처리해주고
    파라미터 튜닝을 진행해주었다.

    LSTM에 dropout을 사용하여 주려고 했으나, 하이퍼파라미터 수정과정에서
    없는 편이 성능이 좋아서 그냥 사용하지 않음

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding,Conv1D, MaxPooling1D,Flatten
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
#코드 출처 - https://wikidocs.net/44249
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)
threshold = 3
total_cnt = len(tokenizer.word_index)
rare_cnt = 0 
total_freq = 0
rare_freq = 0 
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

vocab_size = total_cnt - rare_cnt + 1 
print('단어 집합의 크기 :',vocab_size)
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(x_train)

x_train = tokenizer.texts_to_sequences(X_train)
x_test = tokenizer.texts_to_sequences(x_test)
x_val = tokenizer.texts_to_sequences(X_val)

단어 집합의 크기 : 17630


AttributeError: ignored

In [ ]:
def below_threshold_len(max_len, nested_list):
    cnt = 0
    for s in nested_list:
        if(len(s) <= max_len):
            cnt = cnt + 1
    print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

max_len = 250
below_threshold_len(max_len, x_train)

전체 샘플 중 길이가 250 이하인 샘플의 비율: 91.39852259922374


In [ ]:
y_train = to_categorical(y_train) # 원-핫 인코딩
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)

In [ ]:
X_train = pad_sequences(x_train, maxlen=max_len) 
X_test = pad_sequences(x_test, maxlen=max_len) 

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=0)
X_train_over,y_train_over = smote.fit_sample(x_train,y_train)

ValueError: ignored

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(LSTM(150))
model.add(Dense(7, activation='softmax'))
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=2, save_best_only=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
history = model.fit(X_train, y_train, 
                    batch_size=80, epochs=30, 
                    callbacks=[es, mc], validation_split=0.2)

Epoch 1/30
107/107 [==============================] - ETA: 0s - loss: 1.6020 - acc: 0.4015
Epoch 00001: val_acc improved from -inf to 0.57042, saving model to best_model.h5
107/107 [==============================] - 89s 827ms/step - loss: 1.6020 - acc: 0.4015 - val_loss: 1.2869 - val_acc: 0.5704
Epoch 2/30
107/107 [==============================] - ETA: 0s - loss: 0.8593 - acc: 0.7326
Epoch 00002: val_acc improved from 0.57042 to 0.71408, saving model to best_model.h5
107/107 [==============================] - 87s 818ms/step - loss: 0.8593 - acc: 0.7326 - val_loss: 0.9274 - val_acc: 0.7141
Epoch 3/30
107/107 [==============================] - ETA: 0s - loss: 0.4788 - acc: 0.8586
Epoch 00003: val_acc did not improve from 0.71408
107/107 [==============================] - 88s 822ms/step - loss: 0.4788 - acc: 0.8586 - val_loss: 0.9683 - val_acc: 0.7108
Epoch 4/30
107/107 [==============================] - ETA: 0s - loss: 0.2733 - acc: 0.9252
Epoch 00004: val_acc did not improve from 0.714

In [ ]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))
from sklearn.metrics import classification_report
predictions = loaded_model.predict_classes(X_test)
print(classification_report(np.argmax(y_test, axis = 1),predictions))

37/37 [==============================] - 3s 82ms/step - loss: 0.8372 - acc: 0.7472

 테스트 정확도: 0.7472
              precision    recall  f1-score   support

           0       0.78      0.79      0.78       152
           1       0.77      0.73      0.75       142
           2       0.73      0.56      0.64       148
           3       0.67      0.82      0.74       141
           4       0.68      0.65      0.67       144
           5       0.77      0.81      0.79        93
           6       0.79      0.81      0.80       335

    accuracy                           0.75      1155
   macro avg       0.74      0.74      0.74      1155
weighted avg       0.75      0.75      0.75      1155



In [ ]:
def classif(prediction, wd, y_test):
    y_pred = []
    y_true = []
    dw = {}
    for i in wd:
        dw[i[1]] = dw[i][0]]
    for i in prediction:
        y_pred.append(dw[i[0]])
    for y

SyntaxError: ignored